<a href="https://colab.research.google.com/github/aaryanrajput26/Deep-Learning/blob/main/DL_Exp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [21]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip


--2026-02-08 18:20:11--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 72.247.96.197, 2600:1406:5400:2ae::317f, 2600:1406:5400:2ac::317f
Connecting to download.microsoft.com (download.microsoft.com)|72.247.96.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M  62.5MB/s    in 12s     

2026-02-08 18:20:23 (67.6 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]



In [22]:
!unzip -q kagglecatsanddogs_5340.zip


In [23]:
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            with open(fpath, "rb") as fobj:
                if b"JFIF" not in fobj.peek(10):
                    os.remove(fpath)
                    num_skipped += 1
        except:
            os.remove(fpath)
            num_skipped += 1

print("Deleted:", num_skipped)


Deleted: 1590


In [24]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()
])

dataset = ImageFolder("PetImages", transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

print("Total Images:", len(dataset))


Total Images: 23410


In [25]:
from torch.utils.data import random_split

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset,[train_size,val_size])

train_loader_cd = DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader_cd = DataLoader(val_dataset,batch_size=32)


In [26]:
import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self, activation, num_classes):
        super().__init__()

        act_dict={
            "relu":nn.ReLU(),
            "tanh":nn.Tanh(),
            "leakyrelu":nn.LeakyReLU()
        }
        act_fn=act_dict[activation]

        self.features=nn.Sequential(
            nn.Conv2d(3,32,3,padding=1),
            nn.BatchNorm2d(32),
            act_fn,
            nn.MaxPool2d(2),

            nn.Conv2d(32,64,3,padding=1),
            nn.BatchNorm2d(64),
            act_fn,
            nn.MaxPool2d(2)
        )

        self.classifier=nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*16*16,128),
            act_fn,
            nn.Dropout(0.5),
            nn.Linear(128,num_classes)
        )

    def forward(self,x):
        return self.classifier(self.features(x))


In [27]:
def init_weights(model,method):
    for m in model.modules():
        if isinstance(m,(nn.Conv2d,nn.Linear)):
            if method=="xavier":
                nn.init.xavier_uniform_(m.weight)
            elif method=="kaiming":
                nn.init.kaiming_uniform_(m.weight)
            elif method=="random":
                nn.init.uniform_(m.weight)


In [28]:
import torch

def get_optimizer(name,model):
    if name=="sgd":
        return torch.optim.SGD(model.parameters(),lr=0.01)
    elif name=="adam":
        return torch.optim.Adam(model.parameters(),lr=0.001)
    elif name=="rmsprop":
        return torch.optim.RMSprop(model.parameters(),lr=0.001)


In [29]:
def train_and_evaluate(model,train_loader,val_loader,optimizer,epochs=5):
    criterion=nn.CrossEntropyLoss()
    device="cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    for epoch in range(epochs):
        model.train()
        for images,labels in train_loader:
            images,labels=images.to(device),labels.to(device)
            optimizer.zero_grad()
            outputs=model(images)
            loss=criterion(outputs,labels)
            loss.backward()
            optimizer.step()

    # Evaluation
    model.eval()
    correct=0
    total=0
    with torch.no_grad():
        for images,labels in val_loader:
            images,labels=images.to(device),labels.to(device)
            outputs=model(images)
            _,predicted=torch.max(outputs,1)
            total+=labels.size(0)
            correct+=(predicted==labels).sum().item()

    acc=100*correct/total
    return acc


In [ ]:
activations=["relu","tanh","leakyrelu"]
inits=["xavier","kaiming","random"]
optimizers=["sgd","adam","rmsprop"]

best_acc=0
best_config=None
best_model=None

for act in activations:
    for init in inits:
        for opt in optimizers:

            model=CNNModel(act,num_classes=2)
            init_weights(model,init)
            optimizer=get_optimizer(opt,model)

            acc=train_and_evaluate(model,train_loader_cd,val_loader_cd,optimizer)

            print(act,init,opt,"Accuracy:",acc)

            if acc>best_acc:
                best_acc=acc
                best_config=(act,init,opt)
                best_model=model


In [ ]:
torch.save(best_model.state_dict(),"best_cnn_catsdogs.pth")
print("Best:",best_config,best_acc)
